# Malicous Domain Dataset Preprocessing

In [20]:
import pandas as pd

datasource = "dataset.csv"

data = pd.read_csv(datasource)

#print(data.info())

country_mismatch = 0

print(data.isnull().sum())

country = data.apply(lambda x : True
            if x['CountryCode'] == x["RegisteredCountry"] else False, axis = 1)

print("Count of rows with CountryCode = RegisteredCountry: %d" % (len(country[country == True].index)))

Domain                     0
DNSRecordType              0
MXDnsResponse              0
TXTDnsResponse             0
HasSPFInfo                 0
HasDkimInfo                0
HasDmarcInfo               0
Ip                         0
DomainInAlexaDB            0
CommonPorts                0
CountryCode            29052
RegisteredCountry      77774
CreationDate               0
LastUpdateDate             0
ASN                        0
HttpResponseCode           0
RegisteredOrg          35391
SubdomainNumber            0
Entropy                    0
EntropyOfSubDomains        0
StrangeCharacters          0
TLD                      170
IpReputation               0
DomainReputation           0
ConsoantRatio              0
NumericRatio               0
SpecialCharRatio           0
VowelRatio                 0
ConsoantSequence           0
VowelSequence              0
NumericSequence            0
SpecialCharSequence        0
DomainLength               0
Class                      0
dtype: int64
C

## Remove Rows With Null TLD

In [24]:
data["TLD"].isnull()

RangeIndex(start=0, stop=90000, step=1)